<a href="https://colab.research.google.com/github/LNshuti/estimate-merger-synergies/blob/main/process_records.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from pinecone import PineconeClient
from langchain.prompts import load_prompt
import pinecone
from langchain import OpenAI, ConversationChain
import openai
import os
from unstructured.partition.auto import partition

# Initialize the Pinecone client with a project name
#pinecone_client = PineconeClient()

In [ ]:
import pandas as pd 
from sqlalchemy import create_engine
import psycopg2

In [34]:
import os
# os.environ["OPENAI_API_KEY"] = "..."
openai.api_key = os.environ["OPENAI_API_KEY"]

llm = OpenAI(temperature=0, openai_api_key=openai.api_key)
conversation = ConversationChain(llm=llm, verbose=True)

pinecone.init(api_key=openai.api_key, environment="us-central1-gcp")

In [35]:
conn_string = "host='localhost' dbname='postgres' user='postgres' password='@55!yHSPH'"
conn = psycopg2.connect(conn_string)

query = """select table_name from information_schema.tables where table_schema = 'public';"""
a_df = pd.read_sql_query(query, conn)
a_df.head()

C:\Users\Leonce Nshuti\AppData\Local\Temp\ipykernel_3600\2286687222.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  a_df = pd.read_sql_query(query, conn)


,table_name


In [36]:
cs_loader = UnstructuredPDFLoader("data/csg-ar-2022-en.pdf")
ubs_loader = UnstructuredPDFLoader("data/ubs-20f-full-report=sec-2022.pdf")

data_cs = cs_loader.load()
data_ubs = ubs_loader.load()
print (f'You have {len(data_cs)} document(s) in your data')
print (f'There are {len(data_cs[0].page_content)} characters in your document')

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


In [ ]:
ubs_loader = UnstructuredPDFLoader("data/ubs_20f-full-report-sec-2022.pdf")

data_ubs = ubs_loader.load()

In [ ]:
print (f'You have {len(data_ubs)} document(s) in your data')

In [ ]:
from urllib.parse import quote_plus as urlquote
engine = create_engine("postgresql://postgres:%s@localhost/postgres" % urlquote("@55!yHSPH"))
crs_df = pd.read_sql(query, engine)
crs_df.head()

In [ ]:
print (f'You have {len(data_ubs)} document(s) in your data')
print (f'There are {len(data_ubs[0].page_content)} characters in your document')

Chunk your data up into smaller documents

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data_ubs)
print (f'Now you have {len(texts)} documents')

Create embeddings of your documents to get ready for semantic search

In [ ]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

In [28]:
embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)

In [31]:
# initialize pinecone
pinecone.init(
    api_key="82f63bcf-312f-4b33-a8c4-89c9997c5016",  # find at app.pinecone.io
    environment= "us-central1-gcp" # next to api key in console
)
index_name = "langchain2"

In [32]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

AuthenticationError: Incorrect API key provided: sk-aZ67C***************************************aBWp. You can find your API key at https://platform.openai.com/account/api-keys.

In [ ]:
query = "Using the available information, outline a plan to improve key health indicators in the records. Use generalizable advice as a nutritionist and wellness coach. Provide details for a 6 months plan. Use bullet points."
docs = docsearch.similarity_search(query, include_metadata=True)

In [ ]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

Query those docs to get your answer back


In [ ]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")
print(chain.run(input_documents=docs, question=query))

In [ ]:
from sympy import symbols, Eq, Function

# Current formalism
F, G, m1, m2, r = symbols('F G m1 m2 r')
current_formalism = Eq(F, G * m1 * m2 / r**2)

# New lines of conversation
# (This part is a text-based discussion, so it remains the same)

# New formalism
g, eta, h = symbols('g eta h')
mu, nu = symbols('mu nu', integer=True)
g_munu = Function('g')(mu, nu)
eta_munu = Function('eta')(mu, nu)
h_munu = Function('h')(mu, nu)
new_formalism = Eq(g_munu, eta_munu + h_munu)

print("Current formalism:")
print(current_formalism)
print("\nNew lines of conversation:")
print("Human: How does gravity affect time in general relativity?")
print("AI: In general relativity, gravity is a curvature of spacetime caused by mass. This curvature affects not only the path of objects moving through space but also the flow of time. The closer an object is to a massive body, the slower time flows relative to an observer farther away from the massive body.")
print("\nNew formalism:")
print(new_formalism)
